In [5]:
import numpy as np
import random
import scipy.linalg
import gym
import sys
import os
sys.path.append("../utility")
sys.path.append("../train")
from Utility import DerivativeLiftFunc, data_collecter,RBFLiftFunc

In [6]:
ENV = ["DampingPendulum","Pendulum-v1","CartPole-v1","MountainCarContinuous-v0","Franka","DoublePendulum"]

In [ ]:
def evalKoopmanD(env_index,sample):
    env_name = ENV[env_index]
    print(env_name)
    random.seed(1)
    np.random.seed(1)
    Data_collecter = data_collecter(env_name)
    Nstate = Data_collecter.Nstates
    udim = Data_collecter.udim
    Nrbf = 50
    LiftFunc = RBFLiftFunc(env_name,Nstate,udim,Nrbf,Data_collecter.observation_space)
    NKoopman = LiftFunc.NKoopman
    test_Samples = 5000
    Steps = 30
    train_data = Data_collecter.collect_koopman_data(sample*15,1)
    X_lift = LiftFunc.Psi_su(train_data[0,:,udim:],train_data[0,:,:udim])
    Y_lift = LiftFunc.Psi_su(train_data[1,:,udim:],train_data[0,:,:udim])
    XX = np.dot(X_lift.T,X_lift)
    XY = np.dot(X_lift.T,Y_lift)
    Kd = np.dot(scipy.linalg.pinv(XX),XY).T  
    np.savez("../Prediction_Results/Samples/"+"Kd_"+env_name+"_KoopmanRBFsamples{}".format(sample)+".npz",Kd=Kd,Center=LiftFunc.center)  
    times = 4 
    max_loss_all = np.zeros((times,Steps))
    mean_loss_all = np.zeros((times,Steps))
    for t in range(times):
        test_data_path = "../Data/{}{}.npy".format(env_name,t)
        if os.path.exists(test_data_path):
            test_data = np.load("../Data/{}{}.npy".format(env_name,t))
        else:
            test_data = Data_collecter.collect_koopman_data(test_Samples,Steps)
            np.save("../Data/{}{}.npy".format(env_name,t),test_data)
        X_current = test_data[0,:,:]
        X_current_lift = np.zeros((test_Samples,NKoopman+udim))
        maxloss_list = []
        meanloss_list = []
        for i in range(test_Samples):
            X_current_lift[i] = LiftFunc.Psi_su(X_current[i,udim:],X_current[i,:udim])
        for i in range(Steps):
            X_current_lift = np.dot(X_current_lift,Kd.T)
            X_current_lift[:,NKoopman:] = test_data[i+1,:,:udim]
            Err = X_current_lift[:,:Nstate] - test_data[i+1,:,udim:]
            maxloss = np.mean(np.max(abs(Err),axis=0))
            meanloss = np.mean(np.mean(abs(Err),axis=0))
            maxloss_list.append(maxloss)
            meanloss_list.append(meanloss)
        max_loss_all[t] = np.array(maxloss_list).reshape(-1)
        mean_loss_all[t] = np.array(meanloss_list).reshape(-1)
    max_mean = np.mean(max_loss_all,axis=0)
    max_std = np.std(max_loss_all,axis=0)
    mean_mean =  np.mean(mean_loss_all,axis=0)
    mean_std =  np.std(mean_loss_all,axis=0)  
    np.save("../Prediction_Results/Samples/"+env_name+"_KoopmanRBFsamples{}".format(sample)+".npy",np.array([max_mean,max_std,mean_mean,mean_std]))
    return max_mean,max_std,mean_mean,mean_std 

In [8]:
for sample in [200]:
     evalKoopmanD(0,sample)

DampingPendulum


UnboundLocalError: local variable 'i' referenced before assignment